In [38]:
# Cellule 1 : Imports basiques
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

In [39]:
# Cellule 2 : Chargement et préparation optimisée v2
print("Chargement des données...")
df = pd.read_csv('data_processed/allocine_cleaned.csv')

# Features de base optimisées
X = pd.DataFrame()

# Features principales
X['nb_actors'] = df['actors'].str.count(',') + 1
X['length'] = df['length'].fillna(df['length'].median())

# Périodes optimisées
df['date'] = pd.to_datetime(df['date'])
X['is_good_period'] = df['date'].dt.month.isin([8,9]).astype(int)  # Meilleurs mois
X['is_bad_period'] = df['date'].dt.month.isin([11,12]).astype(int) # Mois difficiles

# Genres avec interactions
def has_genre(x, genre):
    try:
        genres = eval(x)
        return int(genre in genres)
    except:
        return 0

for genre in ['Drame', 'Action', 'Aventure']:
    X[f'has_{genre}'] = df['genre'].apply(lambda x: has_genre(x, genre))

# Interactions améliorées
X['good_period_adventure'] = X['is_good_period'] * X['has_Aventure']
X['actors_length_ratio'] = X['nb_actors'] / X['length']  # Densité d'acteurs
X['drama_actors'] = X['has_Drame'] * X['nb_actors']  # Important vu l'importance du drame

# Target avec transformation log
y = np.log1p(df['french_boxoffice'])

# Debug prints
print("\nStatistiques des entrées (log):")
print(y.describe())

print("\nDistribution des caractéristiques:")
print(f"Films en bonne période (Août-Sept): {X['is_good_period'].sum()}")
print(f"Films en période difficile (Nov-Déc): {X['is_bad_period'].sum()}")

print("\nDistribution des genres:")
for genre in ['Drame', 'Action', 'Aventure']:
    count = X[f'has_{genre}'].sum()
    print(f"{genre}: {count} films")

print("\nStatistiques des features numériques:")
print(X[['nb_actors', 'length', 'actors_length_ratio']].describe())

# Split des données
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("\nTaille des sets:")
print(f"Training: {X_train.shape}")
print(f"Test: {X_test.shape}")

Chargement des données...

Statistiques des entrées (log):
count    7292.000000
mean       11.924368
std         1.629329
min         8.778172
25%        10.680562
50%        11.919803
75%        13.124052
max        16.827512
Name: french_boxoffice, dtype: float64

Distribution des caractéristiques:
Films en bonne période (Août-Sept): 1166
Films en période difficile (Nov-Déc): 1263

Distribution des genres:
Drame: 3187 films
Action: 841 films
Aventure: 698 films

Statistiques des features numériques:
         nb_actors       length  actors_length_ratio
count  7292.000000  7292.000000          7292.000000
mean     13.674849   104.520433             0.128902
std       9.091951    18.952322             0.081633
min       1.000000    24.000000             0.005988
25%       6.000000    93.000000             0.060345
50%      12.000000   102.000000             0.119565
75%      20.000000   114.000000             0.181818
max      40.000000   272.000000             0.435294

Taille des sets

In [40]:
# Cellule 3 : Modèle de régression optimisé
model = RandomForestRegressor(
    n_estimators=300,    # Plus d'arbres
    max_depth=10,        # Profondeur moyenne
    min_samples_leaf=12, # Plus conservateur
    random_state=42
)

print("Entraînement du modèle...")
model.fit(X_train, y_train)

Entraînement du modèle...


RandomForestRegressor(max_depth=10, min_samples_leaf=12, n_estimators=300,
                      random_state=42)

In [41]:
# Cellule 4 : Évaluation avec transformation inverse
y_pred = model.predict(X_test)

# Transformation inverse pour les métriques
y_test_orig = np.expm1(y_test)
y_pred_orig = np.expm1(y_pred)

# Métriques globales
r2 = r2_score(y_test_orig, y_pred_orig)
rmse = np.sqrt(mean_squared_error(y_test_orig, y_pred_orig))
mae = mean_absolute_error(y_test_orig, y_pred_orig)

print("\n=== PERFORMANCE GLOBALE DU MODÈLE ===")
print(f"R² Score: {r2:.4f}")
print(f"RMSE: {rmse:.0f} entrées")
print(f"MAE: {mae:.0f} entrées")

print("\n=== IMPORTANCE DES FEATURES ===")
importances = pd.DataFrame({
    'feature': X.columns,
    'importance': model.feature_importances_
}).sort_values('importance', ascending=False)
print(importances)

print("\n=== EXEMPLES D'UTILISATION ===")
# Exemple 1: Film d'action standard en période normale
example1 = pd.DataFrame({
    'nb_actors': [10],
    'length': [120],
    'is_good_period': [0],
    'is_bad_period': [0],
    'has_Drame': [0],
    'has_Action': [1],
    'has_Aventure': [0],
    'good_period_adventure': [0],
    'actors_length_ratio': [10/120],
    'drama_actors': [0]
})

# Exemple 2: Film d'aventure avec grand casting en bonne période
example2 = pd.DataFrame({
    'nb_actors': [20],
    'length': [150],
    'is_good_period': [1],
    'is_bad_period': [0],
    'has_Drame': [0],
    'has_Action': [0],
    'has_Aventure': [1],
    'good_period_adventure': [1],
    'actors_length_ratio': [20/150],
    'drama_actors': [0]
})

print("1. Pour un film d'action standard (120min, 10 acteurs) en période normale:")
pred1 = np.expm1(model.predict(example1)[0])
print(f"Prédiction: {pred1:.0f} entrées")

print("\n2. Pour un film d'aventure ambitieux (150min, 20 acteurs) en bonne période:")
pred2 = np.expm1(model.predict(example2)[0])
print(f"Prédiction: {pred2:.0f} entrées")

# Analyse des erreurs par période
print("\n=== ANALYSE DES ERREURS ===")
errors = abs(y_test_orig - y_pred_orig)
print("\nDistribution des erreurs:")
print(pd.Series(errors).describe())

# Analyse par période
print("\nErreur moyenne par période:")
for period, name in [('is_good_period', 'Bonne période (Août-Sept)'),
                    ('is_bad_period', 'Période difficile (Nov-Déc)')]:
    mask = X_test[period] == 1
    if mask.sum() > 0:
        err = mean_absolute_error(y_test_orig[mask], y_pred_orig[mask])
        print(f"{name}: {err:.0f} entrées")

# Analyse par genre et période
print("\nErreur moyenne par genre:")
for genre in ['Drame', 'Action', 'Aventure']:
    mask = X_test[f'has_{genre}'] == 1
    if mask.sum() > 0:
        err = mean_absolute_error(y_test_orig[mask], y_pred_orig[mask])
        print(f"{genre}: {err:.0f} entrées")


=== PERFORMANCE GLOBALE DU MODÈLE ===
R² Score: 0.2689
RMSE: 925220 entrées
MAE: 371659 entrées

=== IMPORTANCE DES FEATURES ===
                 feature  importance
0              nb_actors    0.393090
1                 length    0.176232
8    actors_length_ratio    0.149216
9           drama_actors    0.108993
4              has_Drame    0.078853
6           has_Aventure    0.066672
5             has_Action    0.011190
3          is_bad_period    0.009913
2         is_good_period    0.005588
7  good_period_adventure    0.000252

=== EXEMPLES D'UTILISATION ===
1. Pour un film d'action standard (120min, 10 acteurs) en période normale:
Prédiction: 161672 entrées

2. Pour un film d'aventure ambitieux (150min, 20 acteurs) en bonne période:
Prédiction: 1998405 entrées

=== ANALYSE DES ERREURS ===

Distribution des erreurs:
count    1.459000e+03
mean     3.716593e+05
std      8.475815e+05
min      3.940673e+01
25%      3.963356e+04
50%      1.205390e+05
75%      3.188614e+05
max      1.188